# Lab 3 — clustering

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
from scipy import spatial
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, ResetTool, PanTool, WheelZoomTool, SaveTool
output_notebook()
from bokeh.palettes import Dark2_8

%matplotlib inline
plt.style.use("ggplot")

In [ ]:
with open("tagsCoordinates.pickle", "rb") as f:
    tagsCoord = pickle.load(f, encoding="utf-8")

In [ ]:
from sklearn.cluster import KMeans

twoDir = np.zeros([len(tagsCoord),2])
threeDir = np.zeros([len(tagsCoord),3])
fourDir = np.zeros([len(tagsCoord),4])
fiveDir = np.zeros([len(tagsCoord),5])

i = 0
for tags,coord in tagsCoord.items():
    twoDir[i,0] = coord[0]
    twoDir[i,1] = coord[1]
    i += 1    
    


In [ ]:
def drawPlot(array,color):
    source = ColumnDataSource(
        data={
            "x": [x[0] for x in array ],
            "y": [x[1] for x in array],
            "name": [x for x in tagsCoord.keys()],
            "color": [Dark2_8[x] for x in color],
        })

    hover = HoverTool(
        tooltips=[
            ("Name", "@name"),
        ])
    tools = [hover, ResetTool(), PanTool(), WheelZoomTool(), SaveTool()]

    p = figure(plot_width=960, plot_height=360, tools=tools, title="Mouse over the dots")
    p.circle("x", "y", source=source, size=20, color="color", alpha=0.5)
    return p

In [ ]:
def computeCluster(k,array):
    kmeans = KMeans(n_clusters=k).fit(array)
    color =[]
    for item in array:
        d1 =spatial.distance.euclidean(item,kmeans.cluster_centers_[0])
        d2 =spatial.distance.euclidean(item,kmeans.cluster_centers_[1])
        if d1<d2:
            color.append(0)
        else:
            color.append(2)
    return drawPlot(array,color)

In [ ]:
s=computeCluster(2,twoDir)
show(s,notebook_handle=True)

In [ ]:
# Categorial colors



show(p, notebook_handle=True)